In [1]:
import umls_api
import yaml
from tqdm import tqdm
import json

In [2]:
with open("config.yaml", "r") as stream:
    try:
        PARAM = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

umls_token = PARAM["umls_token"]

In [3]:
#umls_api.get_relation("22396", "RXNORM", umls_token, "CHD", "has_parent")

In [4]:
#umls_api.get_relation("22396", "RXNORM", umls_token, "RO", "has_mechanism_of_action")

In [5]:
#umls_api.get_relation("22396", "RXNORM", umls_token, "RO", "has_physiologic_effect")

In [6]:
done = set()

with open('temp/rxnorm_response.txt','r') as f:
    for line in f:
        object = json.loads(line.strip())
        done.add(object['drug_id'])

In [7]:
drugs = {}
for line in open("temp/may_be_treated_response.tsv", "r"):
    fields = line.strip().split("\t")

    for d in json.loads(fields[1]):
        drugs[d["id"]] = d["name"]


for drug_id in tqdm(drugs):
    if drug_id not in done:
        
        temp = {}
        temp["drug_id"] = drug_id
        temp["name"] = drugs[drug_id]
        temp["parents"] = umls_api.get_relation(drug_id, "RXNORM", umls_token, "CHD", "has_parent")
        temp["moa"] = umls_api.get_relation(drug_id, "RXNORM", umls_token, "RO", "has_mechanism_of_action")
        temp["pe"] = umls_api.get_relation(drug_id, "RXNORM", umls_token, "RO", "has_physiologic_effect")

        with open('temp/rxnorm_response.txt','a') as f:
            f.write(json.dumps(temp) + "\n")

100%|██████████| 2945/2945 [1:46:44<00:00,  2.17s/it]


In [8]:
len(drugs)

2945